In [1]:
import json
import litellm

from litellm import experimental_mcp_client
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from mcp.types import CallToolResult

In [2]:
model = 'ollama_chat/llama3.2:latest'

server_params = StdioServerParameters(
    command="python3",
    args=["./main.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        meta, contents = await session.read_resource("schema://main")

        messages = []
        for content in contents[1]:
            messages = [{"role": "system", "content": f"You are a helpful assistant. Your job is to accept natural language queries and return the SQL query that answers the question. You must only return well formed SQL for sqlite3. You may only use select statments as this is for reporting only. Here is the schema of the database:{content.text}"}]

        # Get tools
        tools = await experimental_mcp_client.load_mcp_tools(session=session, format="openai")
        print("MCP TOOLS: ", tools)
         
         
        messages.append({"role": "user", "content": "List the top three artitsts by record sales"})
        llm_response = await litellm.acompletion(
            model=model,
            messages=messages,
            tools=tools,
            temperature=0.0,
        )
        print("LLM RESPONSE: ", json.dumps(llm_response, indent=4, default=str))
        
        fn: str = llm_response["choices"][0]["message"]["tool_calls"][0].function.name
        args: dict = json.loads(llm_response["choices"][0]["message"]["tool_calls"][0].function.arguments)

        messages.append({"role": "assistant", "tool_calls": json.dumps([{"type": "function", "function": {"name": fn, "arguments": args}}])})

        tool_result: CallToolResult = await session.call_tool(fn, args)
        messages.append({"role": "tool", "content": tool_result.content[0].text})

        print("TOOL_RESULT: ", tool_result)

        print(tool_result.content[0].text)

        messages[0]["content"] = "You are a helpful assistant that can answer questions about the database."

        llm_response = await litellm.acompletion(
            model=model,
            messages=messages,
            tools=tools,
            temperature=0.0,
        )

        print("LLM RESPONSE: ", json.dumps(llm_response, indent=4, default=str))


        print(llm_response["choices"][0]["message"]['content'])


MCP TOOLS:  [{'type': 'function', 'function': {'name': 'query_data', 'description': 'Execute SQL queries safely', 'parameters': {'properties': {'sql': {'title': 'Sql', 'type': 'string'}}, 'required': ['sql'], 'title': 'query_dataArguments', 'type': 'object'}, 'strict': False}}]
LLM RESPONSE:  "ModelResponse(id='chatcmpl-e043c9be-f650-48d0-b5ed-ae767e2f7874', created=1744683841, model='ollama_chat/llama3.2:latest', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='', role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{\"sql\": \"SELECT T1.Name FROM artists AS T1 INNER JOIN albums AS T2 ON T1.ArtistId = T2.ArtistId GROUP BY T1.Name ORDER BY COUNT(T2.AlbumId) DESC LIMIT 3\"}', name='query_data'), id='260cf13e-a431-4cf9-90c3-a6259f3f15c3', type='function')], function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=61, prompt_tokens=1292, total_tokens=13